# Wellcome To My Kernel

# Introduction
Greetings from the Kaggle bot! This is an automatically-generated kernel with starter code demonstrating how to read in the data and begin exploring. If you're inspired to dig deeper, click the blue "Fork Notebook" button at the top of this kernel to begin editing.

*  I have processed some columns to get better meaning of them including the user_agent strings and I have created new csv file containing the parsed UA as it took time in parsing so I did not want others to waste their time on parsing it.
* This project will make you understand lot of stuff & try to understand the problem statement by doing research,understanding each & every feature to play with it.

Problem Statement : Build A Model that can detect the Non Human Traffic present in a website!

# **At end please feadback my kernel.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/bot-detection/ibm_data.csv")
df.head()

In [ ]:
print("The Total num of rows and columns in data is  ",df.shape)

In [ ]:
df.info()

In [ ]:
print("There are some null values in data",df["sec_lvl_domn"].isnull().sum())

In [ ]:
df['page_vw_ts'] = pd.to_datetime(df['page_vw_ts'])

In [ ]:
df['page_vw_ts'].dt.year.head() 

#### Filtering and storing the date in data set

In [ ]:
 df['crm_dt'] = df['page_vw_ts'].dt.date

In [ ]:
df.head()

#### 1.Delete the page_vw_ts column

In [ ]:
df = df.drop('page_vw_ts', 1)

In [ ]:
df.head(3)

## 1 DATA Preprocessing on ip_info Dataset

In [ ]:
df.dropna(subset=['city','st','operating_sys'], inplace=True)
df.device_type.fillna(value='unkown_device', inplace=True)
df.sec_lvl_domn.fillna(value='unkown_domain', inplace=True)
df.dropna(inplace=True)
df.drop(labels=['wk', 'mth', 'yr', 'crm_dt'], axis=1, inplace=True)

### Adding a Bounce_rate Column 

In [ ]:

df["bounce_rate_%"] = ((df.VISIT - df.ENGD_VISIT)/df.VISIT)*100

In [ ]:
df.head(3)

In [ ]:
print("Total of No. Of unique IP's is =",df.ip_addr.nunique())

We can't work on all the ip's that visit our website in a day because many of them just visit only a single time So we try to filter those ip's that has high no. of views or visit too mant times, In rules Of detecting bot it is mentioned that bots show a similar pattern in visiting any website...

In [ ]:
# IP Addres's that has total views greater than 24 in a day
ip_views = pd.DataFrame(df.groupby('ip_addr').VIEWS.sum().sort_values())
unique_ip_address = list(ip_views[ip_views.VIEWS > 24].index)

# Limiting the Dataset to those rows that contain one of the ip's present in unique_ip_address 
new_data = df[df.ip_addr.isin(unique_ip_address)]

# Taking intersection of ip's
#unique_ip_address = list(new_data_details.ip_addr.unique())

# These are the filterd IP's on which we have to find Infomation.
print("No. Of unique ip's {}".format(len(unique_ip_address)))

# Examples of unique ip address
unique_ip_address[:10]

In [ ]:
df.head(3)

In [ ]:
new_data.head()

I create a function that will classify all different variants of each os as the same os for example MICROSOFT_WINDOWS8.1 MICROSOFT_WINXP are to be renamed to MICROSOFT PC
I do the same for the other types of os including mobile phones, I classify the ones not falling into these main types as others

In [ ]:
def shortenos(x):
    #print(x)
    if "microsoft" in x.lower().split("_")[0]:
        x="MICROSOFT PC"
        return x
    elif "windowsphone" in x.lower().split("_")[0]:
        x="WINDOWS MOBILE"
        return x
    elif "windowsmobile" in x.lower().split("_")[0]:
        x="WINDOWS MOBILE"
        return x
    elif "macintosh" in x.lower().split("_")[0]:
        x="MACOS PC"
        return x
    elif "ios" in x.lower().split("_")[0]:
        x="IOS PHONE"
        return x
    elif "android" in x.lower().split("_")[0]:
        x="ANDROID"
        return x
    elif "linux" in x.lower().split("_")[0]:
        x="LINUX"
        return x
    elif x.lower()=="notgiven":
        x="NotGiven"
        return x
    else:
        x="OTHER"
        return x

### Lets groupby the new os column we created and sum the values for VIEWS and VISIT
### This will help us in analysing which of these users are belonging to which OS

In [ ]:
df["os"]=df.operating_sys.apply(shortenos)
os_df=df.groupby(["os"]).sum().reset_index()
os_df

## Show in Plot

In [ ]:
import altair as alt

alt.data_transformers.disable_max_rows()
base=alt.Chart(os_df).mark_bar().encode(
x="os",
y="VISIT",
tooltip=["VISIT"]
)

base2=alt.Chart(os_df).mark_bar().encode(
x="os",
y="VIEWS",tooltip=["VIEWS"]
)
alt.hconcat(base,base2)

In [ ]:
df.user_agent.dropna(inplace=True)

# Installing device_detector

In [ ]:
!pip install device_detector

In [ ]:
from device_detector import SoftwareDetector
def parse_family(x):
    
    return SoftwareDetector(x).parse().client_name()
def parse_os(x):
  
    return SoftwareDetector(x).parse().os_name()

### Here as I stated ill be taking the 400000 as my sample dataset

In [ ]:
sample_df=df[:400000]
sample_df.user_agent.dropna(inplace=True)

In [ ]:
x=sample_df["user_agent"].apply(parse_family)

In [ ]:
sample_df["user_browser"]=x
sample_df.user_agent.dropna(inplace=True)
sample_df["user_os"]=sample_df.user_agent.apply(parse_os)

In [ ]:
sample_df